<a href="https://colab.research.google.com/github/thecrazyphysicist369/Content-Emotion-Model/blob/main/main_Roberta_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tweet-preprocessor 2>/dev/null 1>/dev/null
!pip install transformers


In [3]:
!pip install -q --upgrade ipython==5.5.0
!pip install -q --upgrade ipykernel==4.10
!pip install tensorflow==2.4.1

     |████████████████████████████████| 394.3MB 38kB/s 


In [4]:
import preprocessor as p
import numpy as np 
import pandas as pd 
import keras
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import plotly.graph_objects as go
import plotly.express as px
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
#from kaggle_datasets import KaggleDatasets
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from tqdm import tqdm

/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

##Upload the Dataset
Find the dataset **cleanData.csv** in the repository. This dataset is cleaned and pre-processed. If your dataset is not, please consider cleaning it using the **Text Preprocessing.ipynb**.*italicised text* 

In [5]:
data = pd.read_csv("/content/cleanedData.csv", 
                   dtype={"sentiment": "string", "content": "string"}
                   )


##Mapping the Emotions

In [6]:
#Map the 12+ emotions in the dataset as 12
sent_to_id  = {"empty":0, 
               "sadness":1,
               "enthusiasm":2,
               "neutral":0,
               "worry":4,
               "surprise":5,
               "love":6,
               "fun":7,
               "hate":8,
               "happiness":9,
               "boredom":10,
               "relief":11,
               "anger":12,
               "fear" : 3,
               "joy":7,
               "pos":2,
               "neg":10,
               "other": 0}

data["sentiment_id"] = data['sentiment'].map(sent_to_id)


In [7]:
data

,Unnamed: 0,sentiment,content,clean_content,sentiment_id
0,0,empty,@tiffanylue i know i was listenin to bad habi...,i know i was listenin to bad habit earlier and...,0
1,1,sadness,Layin n bed with a headache ughhhh...waitin o...,Layin n bed with a headache ughhhh waitin on y...,1
2,2,sadness,Funeral ceremony...gloomy friday...,Funeral ceremony gloomy friday,1
3,3,enthusiasm,wants to hang out with friends SOON!,wants to hang out with friends SOON!,2
4,4,neutral,@dannycastillo We want to trade with someone w...,We want to trade with someone who has Houston ...,0
...,...,...,...,...,...
645937,646018,neg,"There's a law protecting unborn eagles, but no...",There s a law protecting unborn eagles but not...,10
645938,646019,other,I am 1 in 3... I have had an abortion #Abortio...,I am 1 in 3 I have had an abortion AbortionOnD...,0
645939,646020,neg,How dare you say my sexual preference is a cho...,How dare you say my sexual preference is a cho...,10
645940,646021,neg,"Equal rights for those 'born that way', no rig...",Equal rights for those born that way no rights...,10


##Encode the dataset

In [8]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data.sentiment_id)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y = onehot_encoder.fit_transform(integer_encoded)

##Split the dataset

In [59]:
X_train, X_test, y_train, y_test = train_test_split(data.clean_content,
                                                    Y, 
                                                    random_state=1995, 
                                                    test_size=0.2, 
                                                    shuffle=True)

#ROBERTA

In [60]:
def regular_encode(texts, tokenizer, maxlen):
    enc_di = tokenizer.batch_encode_plus(
        texts,  
        return_token_type_ids=False,
        padding = True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

def build_model(transformer, max_len):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(13, activation='softmax')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [61]:
AUTO = tf.data.experimental.AUTOTUNE
MODEL = 'roberta-base'

In [62]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
max_len = 512

In [20]:
len(X_test)

129189

In [63]:
X_train = X_train[:100000]
X_test = X_test[:10000]

y_train = y_train[:100000]
y_test = y_test[:10000]

In [42]:
len(X_train)
batch_size=195
Epoch=50

In [64]:
'''this code is crashing at original length of the X_train (~500,000)
code run with X_train ~ 100,000 did not crash, could be a problem of the ram'''
X_train = X_train.tolist()
X_test = X_test.tolist()
X_train_t = regular_encode(X_train, tokenizer, maxlen=max_len)
X_test_t = regular_encode(X_test, tokenizer, maxlen=max_len)

In [66]:
print(X_train_t.shape)
X_train_t

(100000, 100)


array([[    0, 35123,   127, ...,     1,     1,     1],
       [    0,   713,   232, ...,     1,     1,     1],
       [    0, 13368,  5450, ...,     1,     1,     1],
       ...,
       [    0,  4763, 13721, ...,     1,     1,     1],
       [    0,    90, 21210, ...,     1,     1,     1],
       [    0,  7682,   183, ...,     1,     1,     1]])

In [46]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train_t, y_train))
    .repeat()
    .shuffle(1995)
    .batch(batch_size)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_test_t, y_test))
    .batch(batch_size)
    .cache()
    .prefetch(AUTO)
)


In [47]:
transformer_layer = TFAutoModel.from_pretrained(MODEL)
model_roberta_base = build_model(transformer_layer, max_len=max_len)
model_roberta_base.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 512)]             0         
_________________________________________________________________
tf_roberta_model_2 (TFRobert TFBaseModelOutputWithPool 124645632 
_________________________________________________________________
tf.__operators__.getitem_2 ( (None, 768)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 13)                9997      
Total params: 124,655,629
Trainable params: 124,655,629
Non-trainable params: 0
_________________________________________________________________


In [67]:
n_steps = X_train_t.shape[0] // batch_size
print(n_steps)
model_roberta_base.fit(train_dataset,steps_per_epoch=n_steps,validation_data=valid_dataset,epochs=Epoch)

512
Epoch 1/50


ValueError: ignored

###Test

In [ ]:
def get_sentiment2(model,text):
    text = clean_text(text)
    #tokenize
    x_test1 = regular_encode([text], tokenizer, maxlen=max_len)
    test1 = (tf.data.Dataset.from_tensor_slices(x_test1).batch(1))
    #test1
    sentiment = model.predict(test1,verbose = 0)
    sent = np.round(np.dot(sentiment,100).tolist(),0)[0]
    result = pd.DataFrame([sent_to_id.keys(),sent]).T
    result.columns = ["sentiment","percentage"]
    result=result[result.percentage !=0]
    return result

In [ ]:
result =get_sentiment2(model_roberta_base,"Had an absolutely brilliant day ðŸ˜ loved seeing an old friend and reminiscing")
plot_result(result)
result =get_sentiment2(model_roberta_base,"The pain my heart feels is just too much for it to bear. Nothing eases this pain. I can’t hold myself back. I really miss you")
plot_result(result)
result =get_sentiment2(model_roberta_base,"I hate this game so much,It make me angry all the time ")
plot_result(result)